## carga das tabelas

In [1]:
import numpy as np
import pandas as pd
from glob import glob
from functools import reduce

As tabelas vêm empilhadas em arquivos texto. Cada arquivo tem um número de linhas diferente, por isso configuramos cada um:

In [2]:
filenames = sorted(glob('./rust_data/*.asc'))
linhas_por_tabela = [137,137,137,137,128,36,60,81]

As 11 primeiras linhas de cada tabela descrevem metadados da tabela.

In [3]:
metadata_columns = [
    'Bus ID',
    'Purchased Month',
    'Purchased Year',
    '1st replacement Month',
    '1st replacement Year',
    '1st replacement Odometer',
    '2nd replacement Month',
    '2nd replacement Year',
    '2nd replacement Odometer',
    'begin Month',
    'begin Year'
]

Para cada arquivo, lemos uma coluna só, mas depois quebramos em várias colunas redimensionando as séries de acordo com as `linhas_por_tabela` de cada um. Então criamos uma tabela com os ids de cada ônibus como nomes de colunas, e os dados dos odômetros que estão a partir da 12a linha.

In [4]:
original_data = (pd.read_csv(filename, header=None) for filename in filenames)
series = [base.values.reshape(-1, linhas_por_tabela[i]).T for i, base in enumerate(original_data)]

odometer = [pd.DataFrame(s[11:], columns=s[0].astype(str)) for s in series]
o_df = reduce(lambda r, s: r.join(s), odometer).fillna(0)

metadata = [pd.DataFrame(s[0:11].T, columns=metadata_columns) for s in series]
m_df = reduce(lambda r, s: r.append(s), metadata).dropna()

Para facilitar o acesso aos dados de datas, convertemos as colunas separadas em mês e ano em datas concretas, usando o dia 1 para preencher o dia. Assim será fácil calcular o tempo entre a compra e a troca.

In [5]:
for agg in ('Purchased', '1st replacement', '2nd replacement', 'begin'):
  s_month = m_df['{} Month'.format(agg)]
  s_year = 1900 + m_df['{} Year'.format(agg)]
  m_df[agg] = pd.to_datetime({'day': 1, 'month': s_month, 'year': s_year}, errors='coerce')


O último passo é juntar todos os frames em um só, preenchendo com zeros as colunas que não têm linhas suficientes. Então salvamos em parquet para ficar fácil de abrir num outro notebook.

In [6]:
m_df.to_parquet('./rust_data/metadata.parquet', compression='gzip')

In [7]:
o_df.to_parquet('./rust_data/odometers.parquet', compression='gzip')